# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

In [4]:
for elem in obj:
    print('"{:<9} = {}"'.format(elem, obj[elem]))

"home_page = https://github.com/pypa/sampleproject"
"keywords  = sample setuptools development"
"license   = MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [6]:
import pandas as pd
import re

In [7]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

In [11]:
p = re.compile(r'([1-2]\d-\d)')
for line in obj:
    print(p.search(line).group(0))

19-1
20-4
20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [21]:
text = 'Написать регулярное выражение,которое позволит найти номера групп студентов.'
re.split('\W', text)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов',
 '']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [23]:
import numpy as np

In [25]:
recipes = pd.read_csv('./data/recipes_sample.csv')
rnd = np.random.choice(len(recipes), 5, replace=False)
print(f'|{"id":^10}|{"minutes":^10}|')
print(f'{"|":-<22}|')
for i in rnd:
    print(f'|{recipes.id[i]:^10}|{recipes.minutes[i]:^10}|')

|    id    | minutes  |
|---------------------|
|  144190  |    13    |
|  47368   |    35    |
|  134422  |   125    |
|  425977  |    20    |
|  160538  |    25    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [53]:
from bs4 import BeautifulSoup

In [54]:
xml_f = open('./data/steps_sample.xml')
steps = BeautifulSoup(xml_f, 'xml')

In [100]:
def show_info(name, steps, minutes, author_id):
    recipe_steps = '\n'.join([f'{i}. {step.capitalize()}' for i, step in enumerate(steps, 1)])
    return (
        f'"{name.title()}"\n\n'
        + recipe_steps +
        '\n----------'
        f'\nАвтор: {author_id}'
        f'\nСреднее время приготовления: {minutes} минут\n'
    )

recipe_id = 170895
recipe = recipes[recipes.id == recipe_id].iloc[0]
recipe_steps = [step.text for step in steps.recipes.find('id', string=recipe_id).parent.find_all('step')]
print(show_info(
    name=recipe['name'],
    steps=recipe_steps,
    minutes=recipe.minutes,
    author_id=recipe.contributor_id
))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [101]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [105]:
p = re.compile(r'[0-9]{1,}\s(hours|hour|minutes|minute)')

for step in steps.recipes.find('id', string='25082').parent.find_all('step'):
    if p.search(step.text):
        print(p.search(step.text).group(0))

20 minutes
10 minutes
2 hours
10 minutes
20 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [110]:
p = re.compile(r'this[\s\w]{0,},\s{0,1}but')

k = 0
desc = []

for description in recipes.description.fillna(''):
    if p.match(description):
        k += 1
        if len(desc) < 3:
            desc.append(description)

desc = '\n\n'.join(desc)

print(f'количество подходящих рецептов: {k}\n3 примера:\n{desc}')

количество подходящих рецептов: 134
3 примера:
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


In [174]:
def is_not_NaN(string):
    return string == string

p = re.compile(r'this[\s\w]{0,},\s{0,1}but')

k = 0
desc = []

for description in recipes.description:
    if is_not_NaN(description) and p.match(description):
        k += 1
        if len(desc) < 3:
            desc.append(description)

desc = '\n\n'.join(desc)

print(f'количество подходящих рецептов: {k}\n3 примера:\n{desc}')

количество подходящих рецептов: 134
3 примера:
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [113]:
p = re.compile(r'[0-9]{0,} / [0-9]{0,1}')

for step in steps.recipes.find('id', string='72367').parent.find_all('step'):
    if match := p.search(step.text):
        capture = match.group(0)
        new_s = ''.join(capture.split(' '))
        print(step.text.replace(capture, new_s))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
combine apples , 1/3 c


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [114]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/evafeldman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [115]:
from nltk.tokenize import word_tokenize

In [132]:
recipes_steps = []

for step in steps.recipes.find_all('step'):
    recipes_steps.append(step.text)

words = set()

for step in recipes_steps:
    for word in word_tokenize(step):
        if word.isalpha():
            words.add(word.lower())

print(len(words))


14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [134]:
from nltk.tokenize import sent_tokenize

In [148]:
tokenize = lambda x: len(sent_tokenize(x))
recipes.sort_values(
    'description',
    key=lambda x: x.fillna('').apply(tokenize),
    ascending=False
)[:5]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [140]:
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/evafeldman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [145]:
def sent_pos(sentence):
    sentence = sentence.split()
    sent_and_pos = pos_tag(sentence)
    pos = []
    for s, p in sent_and_pos:
        pos.append(p.center(len(s)))
    print(' '.join(pos))
    print(' '.join(sentence))

sentence = recipes.name[recipes.id == 241106].item()
sent_pos(sentence)

   JJ     NNS    IN     NNS    VBP    JJ    CC   JJ   NNS  
eggplant steaks with chickpeas feta cheese and black olives
